# Project 3


In [1]:
#Importing modules

import pandas as pd
import tabula
import yaml
from sqlalchemy import create_engine
from sqlalchemy import inspect
import requests
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
import importlib
import json
import boto3
import re
import numpy as np


#starting local database

#read creds
uploader = DatabaseConnector()
ml_yaml = 'ml_dbs.yaml'
uploader.read_db_creds(ml_yaml)

# initialising and returning an sqlalchemy database engine.

uploading = uploader.init_db_engine_postgresql()



Credentials loaded sucessfully!!
Connection successful!


## RDS Database ETL


### 1. Getting data from the RDS database


In [6]:

## read credentials
connector_rds = DatabaseConnector()
ai_core_yaml = 'db_creds.yaml'
connector_rds.read_db_creds(ai_core_yaml)

## initialising and returning an sqlalchemy database engine.
connection_rds = connector_rds.init_db_engine()

## Reading the data from the RDS database
connector_tables_rds = connector_rds.list_db_tables()





Credentials loaded sucessfully!!
Connection successful!
Tables in the database: ['legacy_store_details', 'dim_card_details', 'legacy_users', 'orders_table']


### 2. Extracting the table to a pandas DataFrame.

In [7]:
user_table_rds = 'legacy_users'

extractor_rds = DataExtractor(connection_rds)

user_df_rds = extractor_rds.read_rds_table(user_table_rds)
print(user_df_rds.head(5))
print(user_df_rds.info())




   index first_name last_name date_of_birth                    company  \
0      0   Sigfried     Noack    1990-09-30         Heydrich Junitz KG   
1      1        Guy     Allen    1940-12-01                    Fox Ltd   
2      2      Harry  Lawrence    1995-08-02  Johnson, Jones and Harris   
3      3     Darren   Hussain    1972-09-23                Wheeler LLC   
4      4      Garry     Stone    1952-12-20                 Warner Inc   

                  email_address  \
0             rudi79@winkler.de   
1  rhodesclifford@henderson.com   
2  glen98@bryant-marshall.co.uk   
3    daniellebryan@thompson.org   
4       billy14@long-warren.com   

                                             address         country  \
0                       Zimmerstr. 1/0\n59015 Gießen         Germany   
1  Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH  United Kingdom   
2                 92 Ann drive\nJoanborough\nSK0 6LR  United Kingdom   
3             19 Robinson meadow\nNew Tracy\nW22 2QG  Un

### 3.Cleaning data

In [8]:

clean_data_rds = DataCleaning()
clean_df_rds = clean_data_rds.clean_user_data(user_df_rds)



Checking for Null values:
index            0.0
first_name       0.0
last_name        0.0
date_of_birth    0.0
company          0.0
email_address    0.0
address          0.0
country          0.0
country_code     0.0
phone_number     0.0
join_date        0.0
user_uuid        0.0
dtype: float64
Deleting rows with more than 4 Null values
       index first_name last_name date_of_birth                       company  \
0          0   Sigfried     Noack    1990-09-30            Heydrich Junitz KG   
1          1        Guy     Allen    1940-12-01                       Fox Ltd   
2          2      Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3          3     Darren   Hussain    1972-09-23                   Wheeler LLC   
4          4      Garry     Stone    1952-12-20                    Warner Inc   
...      ...        ...       ...           ...                           ...   
15315  14913    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316  14994    

### 4. Loading the data
#### uploading the cleaned dataframe into the local database


In [9]:
# Connect with the local database



#load clean rds dataframe into local database with a new table
uploader.upload_to_db(uploading, clean_df_rds, 'dim_users')





Data uploaded successfully to dim_users table.


## PDF data: ETL


### 1.Getting the data from PDF & load in df

In [10]:

connector_pdf = DatabaseConnector()
extractor_pdf = DataExtractor(connector_pdf)

link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf"
extraction_pdf = extractor_pdf.retrieve_pdf_data(link)
print(extraction_pdf.head(5))
print(extraction_pdf.info())

### 2.Clean the pdf df

In [ ]:
clean_pdf = DataCleaning()

clean_df_pdf =clean_pdf.clean_card_data(extraction_pdf)


### 3. load into db

In [ ]:


uploader.upload_to_db(uploading, clean_df_pdf, 'dim_card_details')

## PDF data: API







### 1. Getting Data from API


In [ ]:
headers = {
    'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'
}

number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'

connector_api = DatabaseConnector()

extractor_api = DataExtractor(connector_api)

number_of_stores = extractor_api.list_number_of_stores(number_of_stores_endpoint, headers)
print(f"Number of stores: {number_of_stores}")



### 2. Extracting the table to a pandas DataFrame


In [ ]:
retrieve_store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/2'

stores_df = extractor_api.retrieve_stores_data(retrieve_store_endpoint, headers, 2)



### 3.Cleaning data


In [ ]:

clean_stores_api = DataCleaning()
clean_storesdf_api = clean_stores_api.called_clean_store_data(stores_df)

In [ ]:
import importlib
import data_extraction, data_cleaning,database_utils
importlib.reload(data_extraction)
importlib.reload(database_utils)
importlib.reload(data_cleaning)
from data_extraction import DataExtractor
from database_utils import DatabaseConnector
from data_cleaning import DataCleaning

### 4. Loading the data



In [ ]:
uploader.upload_to_db(uploading, clean_storesdf_api, 'dim_store_details')

## S3 data

### 1. Getting data

In [2]:
connector_s3 = DatabaseConnector()
extractor_s3 = DataExtractor(connector_s3)


### 2. Putting into pd df

In [3]:
bucket = 'data-handling-public'
object_key = "products.csv"
pathway = '/Users/student/AICORE/AWS/Project_3/products.csv'

extraction_s3 = extractor_s3.extract_from_s3(bucket, object_key, pathway)
print(extraction_s3.head(20))
print(extraction_s3.info())

    Unnamed: 0                                      product_name  \
0            0       FurReal Dazzlin' Dimples My Playful Dolphin   
1            1               Tiffany's World Day Out At The Park   
2            2               Tiffany's World Pups Picnic Playset   
3            3          Tiffany's World Wildlife Park Adventures   
4            4                           Cosatto Cosy Dolls Pram   
5            5                      Cocomelon Magnetic Scribbler   
6            6                           Peppa Pig House Felties   
7            7                         Paw Patrol Walkie Talkies   
8            8                       Peppa Pig 3D Walkie Talkies   
9            9              Disney Encanto Magical Casa Madrigal   
10          10                     Disney Encanto Mirabel's Room   
11          11              Disney Encanto Isabela's Garden Room   
12          12               Disney Encanto Antonio's Tree House   
13          13                            Tie-Dy

### 3. Cleaning data

In [4]:
clean_s3 = DataCleaning()

clean_s3 =clean_s3.convert_product_weights(extraction_s3)


ValueError: could not convert string to float: ''

### 4. Loading into local db

In [ ]:
uploader.upload_to_db(uploading, clean_api, 'dim_products')